In [9]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')
import pandas
from datetime import timedelta

In [10]:
from config_paper import paper_key_id, paper_secret_key
from cross_signal import *

In [11]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(paper_key_id, paper_secret_key,base_url)

In [12]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-06-02'

In [13]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:

        #momentum indicators
        #dataframe['willr'] = tab.WILLR(dataframe['high'].values, dataframe['low'].values, dataframe['close'].values)
        dataframe['ultosc'] = tab.ULTOSC(dataframe)
        
        
        dataframe['mfi'] = ta.MFI(dataframe['high'], dataframe['low'], dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        
        dataframe['adx'] = tab.ADX(dataframe)

        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=14)
        dataframe['rocr'] =tab.ROCR(dataframe)
        
        
        dataframe['plusdi'] =tab.PLUS_DI(dataframe)
        dataframe['minusdi'] =tab.MINUS_DI(dataframe)
        
        
        
        
        
        return dataframe

In [14]:
import statistics as sts
def quotelister(quotes):
        #exponential moving averages
        asksizelist=[]
        bidsizelist=[]
        askpricelist=[]
        bidpricelist=[]
        
        
        for quote in quotes:
            if quote.ask_size<50:
                if quote.bid_size<50:
                    asksizelist.append(quote.ask_size)
                    bidsizelist.append(quote.bid_size)
                    bidpricelist.append(quote.bid_size*quote.bid_price)
                    askpricelist.append(quote.ask_size*quote.ask_price)

        # required for graphing
        asksize=sum(asksizelist)
        bidsize=sum(bidsizelist)
        avgaskprice=round(sum(askpricelist)/asksize,2)
        avgbidprice=round(sum(bidpricelist)/bidsize,2)

        return asksize,bidsize,avgbidprice,avgaskprice

In [15]:
account = api.get_account()
print(account.status)
account

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [ ]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [8]:
symbollist=pd.read_csv('nasdaq-100-index-05-06-2020.csv')['Symbol']

In [10]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = symbollist
#assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.05

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)
trade=True

#create dict of buyorder.id, submitted_at
startorder_dict_list=[]


In [11]:
#get starting order by order id, see datetime and get rid
#create corresponding orders

In [ ]:
trade=True
while trade==True:
    #
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currenttime=currenthour*100+currentminute
    currenttimestamp=api.get_clock().timestamp
            
            
    #Manage dict  
    if currenttime <=1230 and currenttime >=650: 
        orderlist=api.list_orders()
        expired_order_list=[]
        
        for i in range(len(startorder_dict_list)):
            if startorder_dict_list[i]['datetime']+datetime.timedelta(seconds=15)<currenttimestamp:
                expired_order_list.append(i)
                try:
                    api.cancel_order(startorder_dict_list[i]['id'])
                    print('cancelled '+startorder_dict_list[i]['symbol'] )
                except:('Fail cancellation')
        
        #remove expired orders after cancellation or expiration
        indexes = expired_order_list
        for index in sorted(indexes, reverse=True):
            del expired_order_list[index]
        
        
        
        
        
        for i in range(assetListLen) :
            try:
                symbol = assetsToTrade[i]

                returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from=startdate, to=startdate).df
                    #api.get_barset(symbol,barTimeframe,limit=150).df
                data=populateindicators(returned_data)
            except:
                print('Issue with data '+ symbol)

        
            
            if data['adx'][-1] > 22 and \
            data['mfi'][-1] < 25 and \
            data['ultosc'][-1] > 25 and \
            data['rsi'][-1] < 33 and \
            data['minusdi'][-1] > 25 and \
            data['minusdi'][-1] > data['plusdi'][-1]:
                buyord=True
                assetlist=api.list_positions()
                quotes=api.polygon.historic_quotes_v2(symbol,date=startdate,reverse=True,limit=600)
                asksize,bidsize,avgbidprice,avgaskprice=quotelister(quotes)
                for asset in assetlist:
                    # if asset is in the list of positions we have already bought, then do not buy again
                    if asset.symbol == symbol:
                        buyord=False
                        print(asset.symbol,asset.qty)

                #if asset is not already bought, then buy             
                if buyord==True:
                    #cashBalance = account.daytrading_buying_power
                    #equity
                    cashBalance = float(account.buying_power)
                    targetPositionSize = int(cashBalance / (data['close'][-1] / positionSizing))
                    buyreturned = api.submit_order(symbol,targetPositionSize,"buy","limit","day",limit_price=avgbidprice-.01)
                    startorder_dict_list.append({'symbol':buyreturned.symbol,'id':buyreturned.id,'datetime':buyreturned.submitted_at,'side':buyreturned.side})
                    print('Buy ',buyreturned)


                
         

        
    
    time.sleep(3)
    #get current time and shut off trading after 12
     
    if currenthour >= 13: 
        trade=False
        print('Good work, Turning off')
    

    
    